## 0. Define a Specification

In the first phase of SDMT, we define a `Specification` that represents the requirements the completed model must meet in order to be acceptable for use in the system into which it will be integrated.

#### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [1]:
import os
from mlte.session import set_context, set_store

store_path = os.path.join(os.getcwd(), "store")
os.makedirs(store_path, exist_ok=True)   # Ensure we are creating the folder if it is not there.

set_context("ns", "OxfordFlower", "0.0.1")
set_store(f"local://{store_path}")

#### Build a `Specification`

In MLTE, we define requirements by constructing a specification (`Spec`). For each property, we define the validations to perform as well.

In [2]:
from mlte.spec.spec import Spec

from mlte.property.fairness.fairness import Fairness
from mlte.property.robustness.robustness import Robustness

from mlte.property.costs import (
    StorageCost,
    PredictingComputeCost,
    PredictingMemoryCost
)

from mlte.measurement.storage import LocalObjectSize
from mlte.measurement.cpu import LocalProcessCPUUtilization
from mlte.measurement.memory import LocalProcessMemoryConsumption

from multiple_accuracy import MultipleAccuracy
from ranksums import RankSums
from multiple_ranksums import MultipleRanksums

spec = Spec(properties={
    Fairness("Important check if model performs well accross different populations"): 
                {"accuracy across gardens": MultipleAccuracy.all_accuracies_more_or_equal_than(0.9)},
    Robustness("Robust against blur and noise"): 
                {"ranksums blur2x8": RankSums.p_value_greater_or_equal_to(0.05/3),
                 "ranksums blur5x8": RankSums.p_value_greater_or_equal_to(0.05/3),
                 "ranksums blur0x8": RankSums.p_value_greater_or_equal_to(0.05/3),
                 "multiple ranksums for clade2": MultipleRanksums.all_p_values_greater_or_equal_than(0.05),
                 "multiple ranksums between clade2 and 3": MultipleRanksums.all_p_values_greater_or_equal_than(0.05),
                },
    StorageCost("Critical since model will be in an embedded device"): 
                    {"model size": LocalObjectSize.value().less_than(3000)},                
    PredictingMemoryCost("Useful to evaluate resources needed when predicting"): 
                    {"predicting memory": LocalProcessMemoryConsumption.value().average_consumption_less_than(0.9)},
    PredictingComputeCost("Useful to evaluate resources needed when predicting"): 
                    {"predicting cpu": LocalProcessCPUUtilization.value().max_utilization_less_than(5.0)}                
    })
spec.save(parents=True, force=True)